In [1]:
import chess 
import chess.pgn
import numpy as np
from random import shuffle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import trange
from sklearn.model_selection import train_test_split

# Data Loader

In [2]:
class GameLoader():
    def __init__(self):
        pass
    def labeling_Win_Loss(self,file_pth):
        W_Win=[1]
        W_label=[]
        W_win,B_win=self.win_loss_classifier(file_pth)
        for i in trange(len(W_win)):
            W_label.append(W_Win)
        B_Win=[0]
        B_label=[]
        for i in trange(len(B_win)):
            B_label.append(B_Win)
        return W_win,W_label,B_win,B_label
    
    def convert_to_tensor(self,W_win,B_win,W_label,B_label,device):
        W_Win_tensor=torch.tensor(W_win,dtype=torch.float32,device=device)
        W_label_tensor=torch.tensor(W_label,dtype=torch.float32,device=device)
        B_win_tensor=torch.tensor(B_win,dtype=torch.float32,device=device)
        B_label_tensor=torch.tensor(B_label,dtype=torch.float32,device=device)
        return W_win_tensor,W_label_tensor,B_win_tensor,B_label_tensor
    
    def win_loss_classifier(self,file_pth):
        pgn_file = open(file_pth)
        W_win = []
        B_win= []
        i = 0
        for i in trange(20000):
            if i > 0:
                game = chess.pgn.read_game(pgn_file)
                if game.headers["Result"] == "1-0":
                    temp1 = self.randPosiGeneratorFromGame(game)
                    W_win+= temp1
                elif game.headers["Result"] == "0-1":
                    temp2 = self.randPosiGeneratorFromGame(game)
                    B_win += temp2
                if game is None:
                    break
            i = i + 1
        pgn_file.close()
        return W_win,B_win
        
    def randPosiGeneratorFromGame(self,game):
        random_positions_array = []
        positions = []
        board = game.board()

        for move in game.mainline_moves():
            if not board.is_capture(move):
                position = board.fen()
                bitstring_position = self.fen_to_bitstring(position)
                positions.append(bitstring_position)

            board.push(move)
            no_capture_posis = len(positions)
        indices = list(range(5, no_capture_posis))
        shuffle(indices)
        selected_indices = indices[:15]
        random_positions_array = [positions[index] for index in selected_indices]
        return random_positions_array

    def fen_to_bitstring(self,fen):
        board = chess.Board(fen)
        bitboard = np.zeros(64*6*2+5)
        piece_idx = {'p': 0, 'n': 1, 'b': 2, 'r': 3, 'q': 4, 'k': 5}
        for i in range(64):
            if board.piece_at(i):
                color = int(board.piece_at(i).color) + 1
                bitboard[(piece_idx[board.piece_at(i).symbol().lower()] + i * 6) * color] = 1
        bitboard[-1] = int(board.turn)
        bitboard[-2] = int(board.has_kingside_castling_rights(True))
        bitboard[-3] = int(board.has_kingside_castling_rights(False))
        bitboard[-4] = int(board.has_queenside_castling_rights(True))
        bitboard[-5] = int(board.has_queenside_castling_rights(False))
        return bitboard

In [3]:
Game_Load=GameLoader()
W_win,W_label,B_win,B_label = Game_Load.labeling_Win_Loss("CCRL-404.[881513].pgn")

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/110670 [00:00<?, ?it/s]

  0%|          | 0/87269 [00:00<?, ?it/s]

# Getting Weights

In [4]:
x=20000
White_win=W_win[0:x]
Black_win=B_win[0:x]
White_label=W_label[0:x]
Black_label=B_label[0:x]

In [5]:
np.shape(White_win)

(20000, 773)

In [6]:
White_win_tensor=torch.tensor(White_win,dtype=torch.float32)
Black_win_tensor=torch.tensor(Black_win,dtype=torch.float32)

C:\Users\MR_X\AppData\Local\Temp\ipykernel_14852\1846430892.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  White_win_tensor=torch.tensor(White_win,dtype=torch.float32)


In [7]:
encoder_data=torch.concatenate((White_win_tensor,Black_win_tensor),axis=0)

In [8]:
len(encoder_data)

40000

In [9]:
class Autoencoders(nn.Module):
    def __init__(self):
        super(Autoencoders,self).__init__()
        self.Layer1encoder=nn.Sequential(
            nn.Linear(773,600),
            nn.ReLU(),
            nn.Linear(600,773)
        )
        self.Layer2encoder=nn.Sequential(
            nn.Linear(773,600),
            nn.ReLU(),
            nn.Linear(600,400),
            nn.ReLU(),
            nn.Linear(400,600),
            nn.ReLU(),
            nn.Linear(600,773)
        )
        self.Layer3encoder=nn.Sequential(
            nn.Linear(773,600),
            nn.ReLU(),
            nn.Linear(600,400),
            nn.ReLU(),
            nn.Linear(400,200),
            nn.ReLU(),
            nn.Linear(200,400),
            nn.ReLU(),
            nn.Linear(400,600),
            nn.ReLU(),
            nn.Linear(600,773)
        )
        self.Layer4encoder=nn.Sequential(
            nn.Linear(773,600),
            nn.ReLU(),
            nn.Linear(600,400),
            nn.ReLU(),
            nn.Linear(400,200),
            nn.ReLU(),
            nn.Linear(200,100),
            nn.ReLU(),
            nn.Linear(100,200),
            nn.ReLU(),
            nn.Linear(200,400),
            nn.ReLU(),
            nn.Linear(400,600),
            nn.ReLU(),
            nn.Linear(600,773)
        )
    def GetWeights(self,input_data):
        Weights=[]
        loss=nn.MSELoss()
        num_epochs=200
        #1st layer
        optimizer1=optim.SGD(self.Layer1encoder.parameters(),lr=0.001)
        for epoch in trange(num_epochs):
            output1=self.Layer1encoder(input_data)
            loss1=loss(output1,input_data)
            optimizer1.zero_grad()
            loss1.backward()
            optimizer1.step()
        print(loss1)
        Weight1=list(self.Layer1encoder.parameters())[0]
        print(np.shape(Weight1))
        Weights.append(Weight1)
        
        #2nd layer
        optimizer2=optim.SGD(self.Layer2encoder.parameters(),lr=0.001)
        for epoch in trange(num_epochs):
            output2=self.Layer2encoder(input_data)
            loss2=loss(output2,input_data)
            optimizer2.zero_grad()
            loss2.backward()
            optimizer2.step()
        print(loss2)
        Weight2=list(self.Layer2encoder.parameters())[2]
        print(np.shape(Weight2))
        Weights.append(Weight2) 
        
        #3rd layer
        optimizer3=optim.SGD(self.Layer3encoder.parameters(),lr=0.001)
        for epoch in trange(num_epochs):
            output3=self.Layer3encoder(input_data)
            loss3=loss(output3,input_data)
            optimizer3.zero_grad()
            loss3.backward()
            optimizer3.step()
        print(loss3)
        Weight3=list(self.Layer3encoder.parameters())[4]
        print(np.shape(Weight3))
        Weights.append(Weight3) 
        
        #4th layer
        optimizer4=optim.SGD(self.Layer4encoder.parameters(),lr=0.001)
        for epoch in trange(num_epochs):
            output4=self.Layer4encoder(input_data)
            loss4=loss(output4,input_data)
            optimizer4.zero_grad()
            loss4.backward()
            optimizer4.step()
        print(loss4)
        Weight4=list(self.Layer4encoder.parameters())[6]
        print(np.shape(Weight4))
        Weights.append(Weight4) 
        
        return Weights

In [10]:
getWeights=Autoencoders()
Weights=getWeights.GetWeights(encoder_data)

  0%|          | 0/200 [00:00<?, ?it/s]

tensor(0.0304, grad_fn=<MseLossBackward0>)
torch.Size([600, 773])


  0%|          | 0/200 [00:00<?, ?it/s]

tensor(0.0290, grad_fn=<MseLossBackward0>)
torch.Size([400, 600])


  0%|          | 0/200 [00:00<?, ?it/s]

tensor(0.0289, grad_fn=<MseLossBackward0>)
torch.Size([200, 400])


  0%|          | 0/200 [00:00<?, ?it/s]

tensor(0.0289, grad_fn=<MseLossBackward0>)
torch.Size([100, 200])


# Model

In [11]:
class Pos2Vec(nn.Module):
    def __init__(self):
        super(Pos2Vec, self).__init__()
        self.weights1 = Weights[0]
        self.weights2 = Weights[1]
        self.weights3 = Weights[2]
        self.weights4 = Weights[3]
        self.encoder = nn.Sequential(
            nn.Linear(773, 600),
            nn.ReLU(),
            nn.Linear(600, 400),
            nn.ReLU(),
            nn.Linear(400, 200),
            nn.ReLU(),
            nn.Linear(200, 100)
        )
        self.encoder[0].weight.data = self.weights1
        self.encoder[2].weight.data = self.weights2
        self.encoder[4].weight.data = self.weights3
        self.encoder[6].weight.data = self.weights4
    
    def forward(self, x):
        return self.encoder(x)

In [91]:
class DeepChess(nn.Module):
    def __init__(self):
        super(DeepChess, self).__init__()
        self.pos2vec1 = Pos2Vec()
        self.pos2vec2 = Pos2Vec()
        self.fc_layers = nn.Sequential(
            nn.Linear(200, 100),
            nn.ReLU(),
            nn.Linear(100, 2)
        )
    
    def forward(self, x1, x2):
        x1 = self.pos2vec1(x1)
        x2 = self.pos2vec2(x2)
        x = torch.cat((x1, x2), dim=1)
        x = self.fc_layers(x)
        return x

# Training

In [92]:
class ChessPositionDataset(Dataset):
    def __init__(self, Dataset,Datalabel):
        self.state1 = Dataset[:,0]
        self.state2 = Dataset[:,1]
        self.state1_label = Datalabel[:,0]
        self.state2_label = Datalabel[:,1]
    
    def __len__(self):
        return len(self.state1)
    
    def __getitem__(self, index):
        return self.state1[index], self.state2[index], self.state1_label[index], self.state2_label[index]

In [93]:
White_win=W_win[20000:40000]
Black_win=B_win[20000:40000]
White_label=W_label[20000:40000]
Black_label=B_label[20000:40000]
for i in range(len(White_win)//2):
    White_win[2*i],Black_win[2*i]=Black_win[2*i],White_win[2*i]
    White_label[2*i],Black_label[2*i]=Black_label[2*i],White_label[2*i]


In [94]:
dataset=list(zip(White_win,Black_win))
Datalabel=list(zip(White_label,Black_label))

In [95]:
dataset_t=torch.tensor(dataset)
dataset_t[:,0].shape

torch.Size([20000, 773])

In [96]:
Train_Dataset, Test_Dataset, Train_Datalabel, Test_Datalabel = train_test_split(dataset_t,
                                                                               Datalabel,
                                                                               test_size=0.1,shuffle=True)

In [97]:
Train_Dataset_t=torch.tensor(Train_Dataset,dtype=torch.float32)
Train_Datalabel_t=torch.tensor(Train_Datalabel,dtype=torch.float32)
Test_Dataset_t=torch.tensor(Test_Dataset,dtype=torch.float32)
Test_Datalabel_t=torch.tensor(Test_Datalabel,dtype=torch.float32)

C:\Users\MR_X\AppData\Local\Temp\ipykernel_14852\2403703623.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Train_Dataset_t=torch.tensor(Train_Dataset,dtype=torch.float32)
C:\Users\MR_X\AppData\Local\Temp\ipykernel_14852\2403703623.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Test_Dataset_t=torch.tensor(Test_Dataset,dtype=torch.float32)


In [98]:
Train_Datalabel

[([0], [1]),
 ([1], [0]),
 ([0], [1]),
 ([0], [1]),
 ([0], [1]),
 ([0], [1]),
 ([0], [1]),
 ([0], [1]),
 ([0], [1]),
 ([1], [0]),
 ([0], [1]),
 ([0], [1]),
 ([1], [0]),
 ([1], [0]),
 ([0], [1]),
 ([0], [1]),
 ([1], [0]),
 ([1], [0]),
 ([0], [1]),
 ([1], [0]),
 ([0], [1]),
 ([0], [1]),
 ([0], [1]),
 ([1], [0]),
 ([0], [1]),
 ([1], [0]),
 ([0], [1]),
 ([1], [0]),
 ([1], [0]),
 ([1], [0]),
 ([0], [1]),
 ([0], [1]),
 ([1], [0]),
 ([1], [0]),
 ([0], [1]),
 ([1], [0]),
 ([1], [0]),
 ([0], [1]),
 ([1], [0]),
 ([0], [1]),
 ([0], [1]),
 ([0], [1]),
 ([1], [0]),
 ([0], [1]),
 ([1], [0]),
 ([0], [1]),
 ([1], [0]),
 ([1], [0]),
 ([0], [1]),
 ([1], [0]),
 ([0], [1]),
 ([1], [0]),
 ([1], [0]),
 ([1], [0]),
 ([0], [1]),
 ([0], [1]),
 ([1], [0]),
 ([1], [0]),
 ([0], [1]),
 ([1], [0]),
 ([1], [0]),
 ([1], [0]),
 ([1], [0]),
 ([1], [0]),
 ([0], [1]),
 ([1], [0]),
 ([1], [0]),
 ([0], [1]),
 ([0], [1]),
 ([1], [0]),
 ([0], [1]),
 ([1], [0]),
 ([1], [0]),
 ([0], [1]),
 ([0], [1]),
 ([0], [1]),
 ([0], [1]),

In [99]:
TrainData=ChessPositionDataset(Train_Dataset_t,Train_Datalabel_t)
TestData=ChessPositionDataset(Test_Dataset_t,Test_Datalabel_t)

In [100]:
len(TrainData)

18000

In [101]:
train_loader = DataLoader(TrainData, batch_size=500, shuffle=True)
test_loader = DataLoader(TestData, batch_size=500)

In [104]:
model= DeepChess()
optimizer=optim.Adam(model.parameters(),lr=0.01)
criterion=nn.CrossEntropyLoss()

num_epochs=1000

for epochs in trange(num_epochs):
    model.train()
    for Win,Loss,Win_label,Loss_label in train_loader:
        optimizer.zero_grad()
        outputs=model(Win,Loss)
        Label=torch.cat((Win_label,Loss_label),dim=1)
        loss=criterion(outputs,Label)
        loss.backward()
        optimizer.step()
        
    if epochs%10==0:
        print(f"epochs:{epochs} loss:{loss}")
        

  0%|          | 0/1000 [00:00<?, ?it/s]

epochs:0 loss:0.0912635400891304
epochs:10 loss:0.0069430735893547535
epochs:20 loss:0.013528837822377682
epochs:30 loss:0.00010343480244046077
epochs:40 loss:0.013671139255166054
epochs:50 loss:0.008644817396998405
epochs:60 loss:0.07328111678361893
epochs:70 loss:1.1288257155683823e-05
epochs:80 loss:4.919391813018592e-06
epochs:90 loss:4.393770325350488e-07
epochs:100 loss:2.870479818284366e-07
epochs:110 loss:1.0514028048191904e-07
epochs:120 loss:4.1530651628818305e-07
epochs:130 loss:3.5762678152195804e-08
epochs:140 loss:9.536726963688125e-09
epochs:150 loss:1.0347145007472136e-07
epochs:160 loss:2.0265508382522057e-08
epochs:170 loss:4.639122437311016e-07
epochs:180 loss:2.8848551991700333e-08
epochs:190 loss:9.059896832752656e-09
epochs:200 loss:2.1457660093204822e-09
epochs:210 loss:-0.0
epochs:220 loss:4.529951080911587e-09
epochs:230 loss:2.43186484283342e-08
epochs:240 loss:1.907347613183674e-09
epochs:250 loss:1.0013569529121469e-08
epochs:260 loss:1.1205660399582484e-08


In [145]:
torch.save(model,"DeepChess_model.pt")
torch.save(optimizer.state_dict,"DeepChess_optimizer.pt")

# Test

In [146]:
model=torch.load("DeepChess_model.pt")

In [134]:
Labels=torch.cat((Test_Datalabel_t[:,0],Test_Datalabel_t[:,1]),dim=1)

In [148]:
Labels.shape

torch.Size([2000, 2])

In [149]:
output=model(Test_Dataset_t[:,0],Test_Dataset_t[:,1])
output=torch.softmax(output,dim=1)
output=torch.tensor(output>0.5, dtype=torch.float32)
correct_samples=(output==Labels).float().sum()
total_samples=4000
accuracy=(correct_samples/total_samples).item()*100
accuracy

C:\Users\MR_X\AppData\Local\Temp\ipykernel_14852\3329565746.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output=torch.tensor(output>0.5, dtype=torch.float32)


88.49999904632568